# 2-Wrangling

This sections aims to deal with outliers and missing data.

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
import pickle

In [6]:
loan_data=pd.read_csv("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\raw\\loan.csv", low_memory=False)

In [7]:
feature_desc=pd.read_excel("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\raw\\LCDataDictionary.xlsx")

In [9]:
loan_data.isnull().sum()

id                                            2260668
member_id                                     2260668
loan_amnt                                           0
funded_amnt                                         0
funded_amnt_inv                                     0
term                                                0
int_rate                                            0
installment                                         0
grade                                               0
sub_grade                                           0
emp_title                                      166969
emp_length                                     146907
home_ownership                                      0
annual_inc                                          4
verification_status                                 0
issue_d                                             0
loan_status                                         0
pymnt_plan                                          0
url                         

In [10]:
loan_data.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
       'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint

There are features in this dataset that are not required, so let's remove examine which ones we need and not, one by one.

But before we do that, we will split the set into train and test splits.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(loan_data.drop('loan_status', axis=1).values, 
                                                    loan_data.loan_status.values,
                                                    test_size=0.25,
                                                    stratify=loan_data.loan_status.values)

It took a while to split the data, so we will save the notebook using dill.

In [12]:
with open('train.pickle', 'wb') as f:
    pickle.dump([X_train, y_train], f)

In [13]:
with open('test.pickle', 'wb') as f:
    pickle.dump([X_test, y_test], f)

Let's check if the target variable has any null values in both the test and training set.

In [14]:
np.sum(loan_data.loan_status.isnull())

0

The target variable does not have any null values.

Let's creat 2 dataframes using the training and testing set and renaming the columns so they are easily identifiable.

In [15]:
loan_train=pd.DataFrame(X_train, columns=loan_data.columns.drop("loan_status"))

In [29]:
loan_test=pd.DataFrame(X_test, columns=loan_data.columns.drop("loan_status"))

We will start by removing columns that have all null values.

In [17]:
null_coumns=loan_train.columns[loan_train.isnull().sum()==len(loan_train)]

In [18]:
null_coumns

Index(['id', 'member_id', 'url'], dtype='object')

We see that following columns can be dropped as they are all null values and are insignificant:

1. 'id'
2. 'member_id'
3. 'url'



In [19]:
loan_train.drop(null_coumns, axis=1,inplace=True)

We see that there are a few columns with a significant number of null values. We will set a threshold of 75%. If column values are more than 75% null, they will be dropped as they will not provide much significance when predicting.

In [20]:
null_columns_75=loan_train.columns[(loan_train.isnull().sum()/len(loan_train))>=0.75]

In [21]:
null_columns_75

Index(['desc', 'mths_since_last_record', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'mths_since_recent_bc_dlq',
       'revol_bal_joint', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      d

Just by looking at the columns, none of them would be significant in predicting lateness.

In [22]:
loan_train.drop(null_columns_75, axis=1,inplace=True)

In [23]:
loan_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,8000,8000,8000,36 months,10.99,261.88,B,B2,Legal Assistant,10+ years,...,50,0,0,11100,8116,7800,0,N,Cash,N
1,3900,3900,3900,36 months,12.99,131.39,C,C1,radiographer,6 years,...,0,1,0,25881,10129,7750,13531,N,Cash,N
2,18000,18000,18000,60 months,7.97,364.72,A,A5,Senior Losn Examiner,10+ years,...,0,0,0,305986,82784,35600,117986,N,Cash,N
3,3500,3500,3500,36 months,15.02,121.37,C,C3,detective,10+ years,...,75,1,0,71470,51208,3500,67970,N,Cash,N
4,22000,22000,21950,36 months,7.62,685.55,A,A3,Aptean,9 years,...,25,0,0,345978,44605,17700,50928,N,Cash,N


Our features are now 104, down from 145, a 27.5% reduction.

We now need to deal with all the missing values. Let's find the columsn with missing values.

In [25]:
columns_nan=loan_train.columns[loan_train.isnull().sum()!=0].values

In [26]:
len(columns_nan)

73

In [27]:
columns_nan

array(['emp_title', 'emp_length', 'annual_inc', 'title', 'zip_code',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_util',
       'total_acc', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
       'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'num_accts_e

There are 73 columns with nan values, we need to deal with them.

1. 'emp_title', Employment title. The entries are strings. If there are Nan's, they will be classified them as "Not Given"
2. 'emp_length',Employment Length. The entries are strings. If there are Nan's, they we will classified them as "Not Given"
3. 'annual_inc', Annual Income. The entries are  floats. If there are Nan's, they will be classified as 0
4. 'title', Purpose Title is just another column of purpose but human readable, so this column will be removed.
5. 'zip_code', Zip code is that of loanee addresses. The entries are strings. Nan entries will be replaced with '000xx'
6. 'dti' is debt-to-income ratio. This is a significant ratio. Entries are floats. Nan's will be replaced with the mean. 

In [30]:
loan_train['emp_title']=loan_train['emp_title'].fillna("Not Given")
loan_test['emp_title']=loan_test['emp_title'].fillna("Not Given")

In [31]:
loan_train['emp_length']=loan_train['emp_length'].fillna("Not Given")
loan_test['emp_length']=loan_test['emp_length'].fillna("Not Given")

In [32]:
loan_train['annual_inc']=loan_train['annual_inc'].fillna(0)
loan_test['annual_inc']=loan_test['annual_inc'].fillna(0)

In [33]:
loan_train.drop('title', inplace=True, axis=1)
loan_test.drop('title', inplace=True, axis=1)

In [34]:
loan_train['zip_code']=loan_train['zip_code'].fillna('000xx')
loan_test['zip_code']=loan_test['zip_code'].fillna('000xx')

In [35]:
loan_train['dti']=loan_train['dti'].fillna(np.mean(loan_train['dti']))
loan_test['dti']=loan_test['dti'].fillna(np.mean(loan_train['dti']))

Let's refresh so we have a less crowded list. As the other columns will require us to see the description what they are precisely.

In [36]:
columns_nan_1=loan_train.columns[loan_train.isnull().sum()!=0].values

In [37]:
len(columns_nan_1)

67

In [38]:
columns_nan_1

array(['delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_util',
       'total_acc', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
       'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num

There are 67 remaining columns, let's start!!

1. 'delinq_2yrs is a counter of how many time a loanee was 30+ days late over the past 2 years. Entries are floats. Nan's will be replaced with 0.
2. 'earliest_cr_line'.This when the borrowers first credit line was opened. Entries are strings with dates(Month-Year). If it's Nan, it is most probable thay they never had a credit line. Nan's will be replaced with 'Jan-1000'.
3. 'inq_last_6mths'. This is  a count of the number of inquiries over the last 6 months. Entries are integers. Nan's will be replaced with 0.
4. 'open_acc' is the number of current open credit lines for the borrower. These are integers and are rarely zero. Nan's will be replaced with the mean.
5. 'pub_rec'is the number of deragotory public record which is a sever case(Bankruptcy, civil court etc). These values are most commonly 0. Entries will be raplced with 0.
6. 'revol_util'. This is revolving account utilization rate in %. Entries are floats. (Example (Credit Card Balance/Credit Limit)). We will replace them the mean.
7. 'total_acc' is the number of credit lines in borrowers file. This has a large range of integers(0-100). But for ease of analysis, we will use the mean.
8. 'last_pymnt_d'. This is a string of the date of the latest payment. We will replace Nan's with 'Jan-1000'.
9. 'next_pymnt_d'. This is a string with the next schedule payment date. Nan's will be replaced with 'Jan-1000'.
10. 'last_credit_pull_d'. This is the most recent month Lending Club pulled a credit check for the borrower. Nan's will be replaced with 'Jan-1000'.

In [39]:
feature_desc[feature_desc['LoanStatNew']=='delinq_2yrs']['Description'].values

array(["The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years"],
      dtype=object)

In [40]:
loan_train['delinq_2yrs']=loan_train['delinq_2yrs'].fillna(0)
loan_test['delinq_2yrs']=loan_test['delinq_2yrs'].fillna(0)

In [41]:
feature_desc[feature_desc['LoanStatNew']=='earliest_cr_line']['Description'].values

array(["The month the borrower's earliest reported credit line was opened"],
      dtype=object)

In [42]:
loan_train['earliest_cr_line']=loan_train['earliest_cr_line'].fillna('Jan-1000')
loan_test['earliest_cr_line']=loan_test['earliest_cr_line'].fillna('Jan-1000')

In [43]:
feature_desc[feature_desc['LoanStatNew']=='inq_last_6mths']['Description'].values

array(['The number of inquiries in past 6 months (excluding auto and mortgage inquiries)'],
      dtype=object)

In [44]:
loan_train['inq_last_6mths']=loan_train['inq_last_6mths'].fillna(0)
loan_test['inq_last_6mths']=loan_test['inq_last_6mths'].fillna(0)

In [45]:
feature_desc[feature_desc['LoanStatNew']=='open_acc']['Description'].values

array(["The number of open credit lines in the borrower's credit file."],
      dtype=object)

In [46]:
loan_train['open_acc']=loan_train['open_acc'].fillna(np.mean(loan_train['open_acc']))
loan_test['open_acc']=loan_test['open_acc'].fillna(np.mean(loan_train['open_acc']))

In [47]:
feature_desc[feature_desc['LoanStatNew']=='pub_rec']['Description'].values

array(['Number of derogatory public records'], dtype=object)

In [48]:
loan_train['pub_rec']=loan_train['pub_rec'].fillna(0)
loan_test['pub_rec']=loan_test['pub_rec'].fillna(0)

In [49]:
feature_desc[feature_desc['LoanStatNew']=='revol_util']['Description'].values

array(['Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.'],
      dtype=object)

In [50]:
loan_train['revol_util']=loan_train['revol_util'].fillna(np.mean(loan_train['revol_util']))
loan_test['revol_util']=loan_test['revol_util'].fillna(np.mean(loan_train['revol_util']))

In [51]:
feature_desc[feature_desc['LoanStatNew']=='total_acc']['Description'].values

array(["The total number of credit lines currently in the borrower's credit file"],
      dtype=object)

In [52]:
loan_train['total_acc']=loan_train['total_acc'].fillna(np.mean(loan_train['total_acc']))
loan_test['total_acc']=loan_test['total_acc'].fillna(np.mean(loan_train['total_acc']))

In [53]:
feature_desc[feature_desc['LoanStatNew']=='last_pymnt_d']['Description'].values

array(['Last month payment was received'], dtype=object)

In [54]:
loan_train['last_pymnt_d']=loan_train['last_pymnt_d'].fillna('Jan-1000')
loan_test['last_pymnt_d']=loan_test['last_pymnt_d'].fillna('Jan-1000')

In [55]:
feature_desc[feature_desc['LoanStatNew']=='next_pymnt_d']['Description'].values

array(['Next scheduled payment date'], dtype=object)

In [56]:
loan_train['next_pymnt_d']=loan_train['next_pymnt_d'].fillna('Jan-1000')
loan_test['next_pymnt_d']=loan_test['next_pymnt_d'].fillna('Jan-1000')

In [57]:
feature_desc[feature_desc['LoanStatNew']=='last_credit_pull_d']['Description'].values

array(['The most recent month LC pulled credit for this loan'],
      dtype=object)

In [58]:
loan_train['last_credit_pull_d']=loan_train['last_credit_pull_d'].fillna('Jan-1000')
loan_test['last_credit_pull_d']=loan_test['last_credit_pull_d'].fillna('Jan-1000')

Again Let's refresh so we have a new set to work with.

In [60]:
columns_nan_2=loan_train.columns[loan_train.isnull().sum()!=0].values

In [61]:
len(columns_nan_2)

57

In [62]:
columns_nan_2

array(['mths_since_last_delinq', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
       'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',
       'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_

1. 'mths_since_last_delinq'. This is the number of months since the borrower was last 30+ days late. Nan's are probably for borrowers who were never 30+ days late and will be replaced with zeros.
2. 'collections_12_mths_ex_med'. This is the number of collections made for payments that were late in the last 12 months. Nan's will be replaced with 0.
3. 'mths_since_last_major_derog'. This is the number of months since there was 90-day or worse rating. Nan's will be replaced with 0.
4. 'acc_now_delinq'. This is the number of accounts that the borrower is delinquent(30+ days late). Most of the results are 0. Nans will be replaced with 0.
5. 'tot_coll_amt'. This is the total collected amount ever owed. Entries are floats represnting dollar amounts. Nans will be replaced with zeros.
6. 'tot_cur_bal' is the total current balance owed on all accounts.Nans will be replaced with 0s.
7. 'open_acc_6m' is the number of open trades in the last 6 months.Nans will be replaced with 0s.
8. 'open_act_il' is the number of currently active installment trades. Nans will be replaced with the mean round to the nearest integer.
9. 'open_il_12m' is the number of installment accounts opened in the last 12 months.Nans will be replaced with the mean round to the nearest integer.
10. 'open_il_24m' is the number of installment accounts opened in the last 12 months.Nans will be replaced with the mean round to the nearest integer.
11. 'mths_since_rcnt_il' number of months since most recent installment account opened.Nans will be replaced with the mean round to the nearest integer.
12. 'total_bal_il'. Current balance of only installment accounts. Nans will be replaced with the mean.
13. 'il_util'. Ratio of current balance to credit limit on installment acounts.Nans will be replaced with the mean.
14. 'open_rv_12m'. Number of revoliving accounts opened in the last 12 months. Nans will be replaced with the mean round to the nearest integer..
15. 'open_rv_24m'. Number of revoliving accounts opened in the last 24 months. Nans will be replaced with the mean round to the nearest integer.
16. 'max_bal_bc'. Maximum current balanced owed on all revolving accounts.Nans will be replaced with the mean.
17. 'all_util'. This is the balance to credit limit ratio on all trades.Nans will be replaced with the mean.

In [63]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_last_delinq']['Description'].values

array(["The number of months since the borrower's last delinquency."],
      dtype=object)

In [64]:
loan_train['mths_since_last_delinq']=loan_train['mths_since_last_delinq'].fillna(0)
loan_test['mths_since_last_delinq']=loan_test['mths_since_last_delinq'].fillna(0)

In [65]:
feature_desc[feature_desc['LoanStatNew']=='collections_12_mths_ex_med']['Description'].values

array(['Number of collections in 12 months excluding medical collections'],
      dtype=object)

In [66]:
loan_train['collections_12_mths_ex_med']=loan_train['collections_12_mths_ex_med'].fillna(0)
loan_test['collections_12_mths_ex_med']=loan_test['collections_12_mths_ex_med'].fillna(0)

In [67]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_last_major_derog']['Description'].values

array(['Months since most recent 90-day or worse rating'], dtype=object)

In [68]:
loan_train['mths_since_last_major_derog']=loan_train['mths_since_last_major_derog'].fillna(0)
loan_test['mths_since_last_major_derog']=loan_test['mths_since_last_major_derog'].fillna(0)

In [69]:
feature_desc[feature_desc['LoanStatNew']=='acc_now_delinq']['Description'].values

array(['The number of accounts on which the borrower is now delinquent.'],
      dtype=object)

In [70]:
loan_train['acc_now_delinq']=loan_train['acc_now_delinq'].fillna(0)
loan_test['acc_now_delinq']=loan_test['acc_now_delinq'].fillna(0)

In [71]:
feature_desc[feature_desc['LoanStatNew']=='tot_coll_amt']['Description'].values

array(['Total collection amounts ever owed'], dtype=object)

In [72]:
loan_train['tot_coll_amt']=loan_train['tot_coll_amt'].fillna(0)
loan_test['tot_coll_amt']=loan_test['tot_coll_amt'].fillna(0)

In [73]:
feature_desc[feature_desc['LoanStatNew']=='tot_cur_bal']['Description'].values

array(['Total current balance of all accounts'], dtype=object)

In [74]:
loan_train['tot_cur_bal']=loan_train['tot_cur_bal'].fillna(0)
loan_test['tot_cur_bal']=loan_test['tot_cur_bal'].fillna(0)

In [75]:
feature_desc[feature_desc['LoanStatNew']=='open_acc_6m']['Description'].values

array(['Number of open trades in last 6 months'], dtype=object)

In [76]:
loan_train['open_acc_6m']=loan_train['open_acc_6m'].fillna(0)
loan_test['open_acc_6m']=loan_test['open_acc_6m'].fillna(0)

In [77]:
feature_desc[feature_desc['LoanStatNew']=='open_act_il']['Description'].values

array(['Number of currently active installment trades'], dtype=object)

In [78]:
loan_train['open_act_il']=loan_train['open_act_il'].fillna(np.round(np.mean(loan_train['open_act_il']),0))
loan_test['open_act_il']=loan_test['open_act_il'].fillna(np.round(np.mean(loan_train['open_act_il']),0))

In [79]:
feature_desc[feature_desc['LoanStatNew']=='open_il_12m']['Description'].values

array(['Number of installment accounts opened in past 12 months'],
      dtype=object)

In [80]:
loan_train['open_il_12m']=loan_train['open_il_12m'].fillna(np.round(np.mean(loan_train['open_il_12m']),0))
loan_test['open_il_12m']=loan_test['open_il_12m'].fillna(np.round(np.mean(loan_train['open_il_12m']),0))

In [81]:
feature_desc[feature_desc['LoanStatNew']=='open_il_24m']['Description'].values

array(['Number of installment accounts opened in past 24 months'],
      dtype=object)

In [82]:
loan_train['open_il_24m']=loan_train['open_il_24m'].fillna(np.round(np.mean(loan_train['open_il_24m']),0))
loan_test['open_il_24m']=loan_test['open_il_24m'].fillna(np.round(np.mean(loan_train['open_il_24m']),0))

In [83]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_rcnt_il']['Description'].values

array(['Months since most recent installment accounts opened'],
      dtype=object)

In [84]:
loan_train['mths_since_rcnt_il']=loan_train['mths_since_rcnt_il'].fillna(np.round(np.mean(loan_train['mths_since_rcnt_il']),0))
loan_test['mths_since_rcnt_il']=loan_test['mths_since_rcnt_il'].fillna(np.round(np.mean(loan_train['mths_since_rcnt_il']),0))

In [85]:
feature_desc[feature_desc['LoanStatNew']=='total_bal_il']['Description'].values

array(['Total current balance of all installment accounts'], dtype=object)

In [86]:
loan_train['total_bal_il']=loan_train['total_bal_il'].fillna(np.mean(loan_train['total_bal_il']))
loan_test['total_bal_il']=loan_test['total_bal_il'].fillna(np.mean(loan_train['total_bal_il']))

In [87]:
feature_desc[feature_desc['LoanStatNew']=='il_util']['Description'].values

array(['Ratio of total current balance to high credit/credit limit on all install acct'],
      dtype=object)

In [88]:
loan_train['il_util']=loan_train['il_util'].fillna(np.mean(loan_train['il_util']))
loan_test['il_util']=loan_test['il_util'].fillna(np.mean(loan_train['il_util']))

In [89]:
feature_desc[feature_desc['LoanStatNew']=='open_rv_12m']['Description'].values

array(['Number of revolving trades opened in past 12 months'],
      dtype=object)

In [ ]:
loan_train['open_rv_12m']=loan_train['open_rv_12m'].fillna(np.round(np.mean(loan_train['open_rv_12m']),0))
loan_test['open_rv_12m']=loan_test['open_rv_12m'].fillna(np.round(np.mean(loan_train['open_rv_12m']),0))

In [ ]:
loan_train['open_rv_24m']=loan_train['open_rv_24m'].fillna(np.round(np.mean(loan_train['open_rv_24m']),0))
loan_test['open_rv_24m']=loan_test['open_rv_24m'].fillna(np.round(np.mean(loan_train['open_rv_24m']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='max_bal_bc']['Description'].values

In [ ]:
loan_train['max_bal_bc']=loan_train['max_bal_bc'].fillna(np.mean(loan_train['max_bal_bc']))
loan_test['max_bal_bc']=loan_test['max_bal_bc'].fillna(np.mean(loan_train['max_bal_bc']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='all_util']['Description'].values

In [ ]:
loan_train['all_util']=loan_train['all_util'].fillna(np.mean(loan_train['all_util']))
loan_test['all_util']=loan_test['all_util'].fillna(np.mean(loan_train['all_util']))

Once again, let's refresh to clear the full columns.

In [ ]:
columns_nan_3=loan_train.columns[loan_train.isnull().sum()!=0].values

In [ ]:
len(columns_nan_3)

In [ ]:
columns_nan_3

1. 'total_rev_hi_lim'. We do not have a desciption but we can assume this is the high limit of all revolinvg accounts. Nans can be replaced with means.
2. 'inq_fi'. Number of personal finance credit inquiries. Nans can be raplaced with the mean rounded to the nearest integer
3. 'total_cu_tl'. Number of finance trades in total.Nans can be raplaced with the mean rounded to the nearest integer.
4. 'inq_last_12m'. This is the number of credit inquireis in the last 12 months. Nans can be raplaced with the mean rounded to the nearest integer.
5. 'acc_open_past_24mths'. Number of trades opened in the last 24 months.Nans can be raplaced with the mean rounded to the nearest integer.
6. 'avg_cur_bal'. This is the average current balance of all accounts. Nans can be replaced with the mean.
7. 'bc_open_to_buy'. This is the total dollar value to buy on revolving bankcards. Nans can be replaced with the mean 
8. 'bc_util'. This is the ratio of total current balance to high credit/credit limit for all bankcard accounts. This can be replaced with the mean.
9. 'chargeoff_within_12_mths' Number of charges off in the last 12 months. Nans can be replced with 0.
10. 'delinq_amnt'. The past-du amount for the accounts that the borrower is now delinquent. Nans can be replaced with 0s.
11. 'mo_sin_old_il_acct'. This the number of months since oldest bank installment account opened.Nans can be raplaced with the mean rounded to the nearest integer.
12. 'mo_sin_old_rev_tl_op'. This is the number of months since oldest revolving account opened. Nans can be raplaced with the mean rounded to the nearest integer.
13. 'mo_sin_rcnt_rev_tl_op'. This is the number of months since the most recent revolving account opened.Nans can be raplaced with the mean rounded to the nearest integer.
14. 'mo_sin_rcnt_tl'. This is the number of months since the most recent account opened. Nans can be raplaced with the mean rounded to the nearest integer.  
15. 'mort_acc'. This is the number of mortgage accounts. Nans can be raplaced with the mean rounded to the nearest integer.

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='total_rev_hi_lim']['Description'].values

In [ ]:
loan_train['total_rev_hi_lim']=loan_train['total_rev_hi_lim'].fillna(np.mean(loan_train['total_rev_hi_lim']))
loan_test['total_rev_hi_lim']=loan_test['total_rev_hi_lim'].fillna(np.mean(loan_train['total_rev_hi_lim']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='inq_fi']['Description'].values

In [ ]:
loan_train['inq_fi']=loan_train['inq_fi'].fillna(np.round(np.mean(loan_train['inq_fi']),0))
loan_test['inq_fi']=loan_test['inq_fi'].fillna(np.round(np.mean(loan_train['inq_fi']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='total_cu_tl']['Description'].values

In [ ]:
loan_train['total_cu_tl']=loan_train['total_cu_tl'].fillna(np.round(np.mean(loan_train['total_cu_tl']),0))
loan_test['total_cu_tl']=loan_test['total_cu_tl'].fillna(np.round(np.mean(loan_train['total_cu_tl']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='inq_last_12m']['Description'].values

In [ ]:
loan_train['inq_last_12m']=loan_train['inq_last_12m'].fillna(np.round(np.mean(loan_train['inq_last_12m']),0))
loan_test['inq_last_12m']=loan_test['inq_last_12m'].fillna(np.round(np.mean(loan_train['inq_last_12m']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='acc_open_past_24mths']['Description'].values

In [ ]:
loan_train['acc_open_past_24mths']=loan_train['acc_open_past_24mths'].fillna(np.round(np.mean(loan_train['acc_open_past_24mths']),0))
loan_test['acc_open_past_24mths']=loan_test['acc_open_past_24mths'].fillna(np.round(np.mean(loan_train['acc_open_past_24mths']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='avg_cur_bal']['Description'].values

In [ ]:
loan_train['avg_cur_bal']=loan_train['avg_cur_bal'].fillna(np.mean(loan_train['avg_cur_bal']))
loan_test['avg_cur_bal']=loan_test['avg_cur_bal'].fillna(np.mean(loan_train['avg_cur_bal']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='bc_open_to_buy']['Description'].values

In [ ]:
loan_train['bc_open_to_buy']=loan_train['bc_open_to_buy'].fillna(np.mean(loan_train['bc_open_to_buy']))
loan_test['bc_open_to_buy']=loan_test['bc_open_to_buy'].fillna(np.mean(loan_train['bc_open_to_buy']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='bc_util']['Description'].values

In [ ]:
loan_train['bc_util']=loan_train['bc_util'].fillna(np.mean(loan_train['bc_util']))
loan_test['bc_util']=loan_test['bc_util'].fillna(np.mean(loan_train['bc_util']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='chargeoff_within_12_mths']['Description'].values

In [ ]:
loan_train['chargeoff_within_12_mths']=loan_train['chargeoff_within_12_mths'].fillna(0)
loan_test['chargeoff_within_12_mths']=loan_test['chargeoff_within_12_mths'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='delinq_amnt']['Description'].values

In [ ]:
loan_train['delinq_amnt']=loan_train['delinq_amnt'].fillna(0)
loan_testn['delinq_amnt']=loan_test['delinq_amnt'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mo_sin_old_il_acct']['Description'].values

In [ ]:
loan_train['mo_sin_old_il_acct']=loan_train['mo_sin_old_il_acct'].fillna(np.round(np.mean(loan_train['mo_sin_old_il_acct']),0))
loan_test['mo_sin_old_il_acct']=loan_test['mo_sin_old_il_acct'].fillna(np.round(np.mean(loan_train['mo_sin_old_il_acct']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mo_sin_old_rev_tl_op']['Description'].values

In [ ]:
loan_train['mo_sin_old_rev_tl_op']=loan_train['mo_sin_old_rev_tl_op'].fillna(np.round(np.mean(loan_train['mo_sin_old_rev_tl_op']),0))
loan_test['mo_sin_old_rev_tl_op']=loan_test['mo_sin_old_rev_tl_op'].fillna(np.round(np.mean(loan_train['mo_sin_old_rev_tl_op']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mo_sin_rcnt_rev_tl_op']['Description'].values

In [ ]:
loan_train['mo_sin_rcnt_rev_tl_op']=loan_train['mo_sin_rcnt_rev_tl_op'].fillna(np.round(np.mean(loan_train['mo_sin_rcnt_rev_tl_op']),0))
loan_test['mo_sin_rcnt_rev_tl_op']=loan_test['mo_sin_rcnt_rev_tl_op'].fillna(np.round(np.mean(loan_train['mo_sin_rcnt_rev_tl_op']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mo_sin_rcnt_tl']['Description'].values

In [ ]:
loan_train['mo_sin_rcnt_tl']=loan_train['mo_sin_rcnt_tl'].fillna(np.round(np.mean(loan_train['mo_sin_rcnt_tl']),0))
loan_test['mo_sin_rcnt_tl']=loan_test['mo_sin_rcnt_tl'].fillna(np.round(np.mean(loan_train['mo_sin_rcnt_tl']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mort_acc']['Description'].values

In [ ]:
loan_train['mort_acc']=loan_train['mort_acc'].fillna(np.round(np.mean(loan_train['mort_acc']),0))
loan_test['mort_acc']=loan_test['mort_acc'].fillna(np.round(np.mean(loan_train['mort_acc']),0))

AND once again....refresh

In [ ]:
columns_nan_4=loan_train.columns[loan_train.isnull().sum()!=0].values

In [ ]:
len(columns_nan_4)

In [ ]:
columns_nan_4

1. 'mths_since_recent_bc'. Number of months since most recent bankcard account opened. Nans replaced with mean round to nearest integer.
2. 'mths_since_recent_inq' Number of months since most recent inquiry. Nans replaced with mean round to nearest integer.
3. 'mths_since_recent_revol_delinq'. Number of months since most recent revolving delinquency.Nans replaced with 0
4. 'num_accts_ever_120_pd'. Number of accounts ever 120 or more days past due. Nans replaced with 0.
5. 'num_actv_bc_tl'. Number of currently active bankcard accounts. Nans replaced with mean round to nearest integer.
6. 'num_actv_rev_tl' Number of currently active revolving trades.Nans replaced with mean round to nearest integer.
7. 'num_bc_sats'.Number of satisfactory bankcard accounts. Nans replaced with mean round to nearest integer.
8. 'num_bc_tl'.Number of bankcard accounts.  Nans replaced with mean round to nearest integer.
9. 'num_il_tl'. Number of installment accounts. Nans replaced with mean round to nearest integer.
10. 'num_op_rev_tl'. Number of open revolving accounts

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_recent_bc']['Description'].values

In [ ]:
loan_train['mths_since_recent_bc']=loan_train['mths_since_recent_bc'].fillna(np.round(np.mean(loan_train['mths_since_recent_bc']),0))
loan_test['mths_since_recent_bc']=loan_test['mths_since_recent_bc'].fillna(np.round(np.mean(loan_train['mths_since_recent_bc']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_recent_inq']['Description'].values

In [ ]:
loan_train['mths_since_recent_inq']=loan_train['mths_since_recent_inq'].fillna(np.round(np.mean(loan_train['mths_since_recent_inq']),0))
loan_test['mths_since_recent_inq']=loan_test['mths_since_recent_inq'].fillna(np.round(np.mean(loan_train['mths_since_recent_inq']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='mths_since_recent_revol_delinq']['Description'].values

In [ ]:
loan_train['mths_since_recent_revol_delinq']=loan_train['mths_since_recent_revol_delinq'].fillna(0)
loan_test['mths_since_recent_revol_delinq']=loan_test['mths_since_recent_revol_delinq'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_accts_ever_120_pd']['Description'].values

In [ ]:
loan_train['num_accts_ever_120_pd']=loan_train['num_accts_ever_120_pd'].fillna(0)
loan_test['num_accts_ever_120_pd']=loan_test['num_accts_ever_120_pd'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_actv_bc_tl']['Description'].values

In [ ]:
loan_train['num_actv_bc_tl']=loan_train['num_actv_bc_tl'].fillna(np.round(np.mean(loan_train['mths_since_recent_inq']),0))
loan_test['num_actv_bc_tl']=loan_test['num_actv_bc_tl'].fillna(np.round(np.mean(loan_train['mths_since_recent_inq']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_actv_rev_tl']['Description'].values

In [ ]:
loan_train['num_actv_rev_tl']=loan_train['num_actv_rev_tl'].fillna(np.round(np.mean(loan_train['num_actv_rev_tl']),0))
loan_test['num_actv_rev_tl']=loan_test['num_actv_rev_tl'].fillna(np.round(np.mean(loan_train['num_actv_rev_tl']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_bc_sats']['Description'].values

In [ ]:
loan_train['num_bc_sats']=loan_train['num_bc_sats'].fillna(np.round(np.mean(loan_train['num_bc_sats']),0))
loan_test['num_bc_sats']=loan_test['num_bc_sats'].fillna(np.round(np.mean(loan_train['num_bc_sats']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_bc_tl']['Description'].values

In [ ]:
loan_train['num_bc_tl']=loan_train['num_bc_tl'].fillna(np.round(np.mean(loan_train['num_bc_tl']),0))
loan_test['num_bc_tl']=loan_test['num_bc_tl'].fillna(np.round(np.mean(loan_train['num_bc_tl']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_il_tl']['Description'].values

In [ ]:
loan_train['num_il_tl']=loan_train['num_il_tl'].fillna(np.round(np.mean(loan_train['num_il_tl']),0))
loan_test['num_il_tl']=loan_test['num_il_tl'].fillna(np.round(np.mean(loan_train['num_il_tl']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_op_rev_tl']['Description'].values

In [ ]:
loan_train['num_op_rev_tl']=loan_train['num_op_rev_tl'].fillna(np.round(np.mean(loan_train['num_op_rev_tl']),0))
loan_test['num_op_rev_tl']=loan_test['num_op_rev_tl'].fillna(np.round(np.mean(loan_train['num_op_rev_tl']),0))

This will be the last refresh of them all.........

In [ ]:
columns_nan_5=loan_train.columns[loan_train.isnull().sum()!=0].values

In [ ]:
len(columns_nan_5)

In [ ]:
columns_nan_5

1. 'num_rev_accts'. This is the number of revolving accounts per borrower. Nans replaced with mean rounded to nearest integer.
2. 'num_rev_tl_bal_gt_0'. Number of revolving trades with balance >0. Nans replaced with mean rounded to nearest integer.
3. 'num_sats'. Number of satisfactoy accounts.  Nans replaced with mean rounded to nearest integer. 
4. 'num_tl_120dpd_2m'. Number of accounts currently 120 days past due (updated in past 2 months). Nans replaced with 0.
5. 'num_tl_30dpd'.Number of accounts currently 30 days past due (updated in past 2 months).Nans replaced with 0.
6. 'num_tl_90g_dpd_24m'. Number of accounts 90 or more days past due in last 24 months. Nans replaced with 0.
7. 'num_tl_op_past_12m'. Number of accounts opened in past 12 months.Nans replaced with mean rounded to nearest integer.
8. 'pct_tl_nvr_dlq'. Percent of trades never delinquent. Nans replaced with mean.
9. 'percent_bc_gt_75'.Percentage of all bankcard accounts > 75% of limit.Nans replaced with mean.
10. 'pub_rec_bankruptcies'.Number of public record bankruptcies. Nans replaced with 0.
11. 'tax_liens'. Number of tax liens. Nans replaced with 0.
12. 'tot_hi_cred_lim'. Total high credit/credit limit. Nans replaced with mean.
13. 'total_bal_ex_mort'. Total credit balance excluding mortgage.Nans replaced with mean.
14. 'total_bc_limit'. Total bankcard high credit/credit limit. Nans replaced with mean.
15. 'total_il_high_credit_limit'. Total installment high credit/credit limit. Nans replaced with mean.

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_rev_accts']['Description'].values

In [ ]:
loan_train['num_rev_accts']=loan_train['num_rev_accts'].fillna(np.round(np.mean(loan_train['num_rev_accts']),0))
loan_test['num_rev_accts']=loan_test['num_rev_accts'].fillna(np.round(np.mean(loan_train['num_rev_accts']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_rev_tl_bal_gt_0']['Description'].values

In [ ]:
loan_train['num_rev_tl_bal_gt_0']=loan_train['num_rev_tl_bal_gt_0'].fillna(np.round(np.mean(loan_train['num_rev_tl_bal_gt_0']),0))
loan_test['num_rev_tl_bal_gt_0']=loan_test['num_rev_tl_bal_gt_0'].fillna(np.round(np.mean(loan_train['num_rev_tl_bal_gt_0']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_sats']['Description'].values

In [ ]:
loan_train['num_sats']=loan_train['num_sats'].fillna(np.round(np.mean(loan_train['num_sats']),0))
loan_test['num_sats']=loan_test['num_sats'].fillna(np.round(np.mean(loan_train['num_sats']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_tl_120dpd_2m']['Description'].values

In [ ]:
loan_train['num_tl_120dpd_2m']=loan_train['num_tl_120dpd_2m'].fillna(0)
loan_test['num_tl_120dpd_2m']=loan_test['num_tl_120dpd_2m'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_tl_30dpd']['Description'].values

In [ ]:
loan_train['num_tl_30dpd']=loan_train['num_tl_30dpd'].fillna(0)
loan_test['num_tl_30dpd']=loan_test['num_tl_30dpd'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_tl_90g_dpd_24m']['Description'].values

In [ ]:
loan_train['num_tl_90g_dpd_24m']=loan_train['num_tl_90g_dpd_24m'].fillna(0)
loan_test['num_tl_90g_dpd_24m']=loan_test['num_tl_90g_dpd_24m'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='num_tl_op_past_12m']['Description'].values

In [ ]:
loan_train['num_tl_op_past_12m']=loan_train['num_tl_op_past_12m'].fillna(np.round(np.mean(loan_train['num_tl_op_past_12m']),0))
loan_test['num_tl_op_past_12m']=loan_test['num_tl_op_past_12m'].fillna(np.round(np.mean(loan_train['num_tl_op_past_12m']),0))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='pct_tl_nvr_dlq']['Description'].values

In [ ]:
loan_train['pct_tl_nvr_dlq']=loan_train['pct_tl_nvr_dlq'].fillna(np.mean(loan_train['pct_tl_nvr_dlq']))
loan_test['pct_tl_nvr_dlq']=loan_test['pct_tl_nvr_dlq'].fillna(np.mean(loan_train['pct_tl_nvr_dlq']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='percent_bc_gt_75']['Description'].values

In [ ]:
loan_train['percent_bc_gt_75']=loan_train['percent_bc_gt_75'].fillna(np.mean(loan_train['percent_bc_gt_75']))
loan_test['percent_bc_gt_75']=loan_test['percent_bc_gt_75'].fillna(np.mean(loan_train['percent_bc_gt_75']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='pub_rec_bankruptcies']['Description'].values

In [ ]:
loan_train['pub_rec_bankruptcies']=loan_train['pub_rec_bankruptcies'].fillna(0)
loan_test['pub_rec_bankruptcies']=loan_test['pub_rec_bankruptcies'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='tax_liens']['Description'].values

In [ ]:
loan_train['tax_liens']=loan_train['tax_liens'].fillna(0)
loan_test['tax_liens']=loan_test['tax_liens'].fillna(0)

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='tot_hi_cred_lim']['Description'].values

In [ ]:
loan_train['tot_hi_cred_lim']=loan_train['tot_hi_cred_lim'].fillna(np.mean(loan_train['tot_hi_cred_lim']))
loan_test['tot_hi_cred_lim']=loan_test['tot_hi_cred_lim'].fillna(np.mean(loan_train['tot_hi_cred_lim']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='total_bal_ex_mort']['Description'].values

In [ ]:
loan_train['total_bal_ex_mort']=loan_train['total_bal_ex_mort'].fillna(np.mean(loan_train['total_bal_ex_mort']))
loan_test['total_bal_ex_mort']=loan_test['total_bal_ex_mort'].fillna(np.mean(loan_train['total_bal_ex_mort']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='total_bc_limit']['Description'].values

In [ ]:
loan_train['total_bc_limit']=loan_train['total_bc_limit'].fillna(np.mean(loan_train['total_bc_limit']))
loan_test['total_bc_limit']=loan_test['total_bc_limit'].fillna(np.mean(loan_train['total_bc_limit']))

In [ ]:
feature_desc[feature_desc['LoanStatNew']=='total_il_high_credit_limit']['Description'].values

In [ ]:
loan_train['total_il_high_credit_limit']=loan_train['total_il_high_credit_limit'].fillna(np.mean(loan_train['total_il_high_credit_limit']))
loan_test['total_il_high_credit_limit']=loan_test['total_il_high_credit_limit'].fillna(np.mean(loan_train['total_il_high_credit_limit']))

Let's do a final test to ensure that there are no Nan values in our data.

In [ ]:
loan_train.columns[loan_train.isnull().sum()!=0]

In [ ]:
loan_test.columns[loan_train.isnull().sum()!=0]

All our columns contating values and there were no evident outliers detected that were signicant enough to remove.

In [ ]:
df_X_train=loan_train
df_X_test=loan_test

In [ ]:
df_X_train.to_csv("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\processed\\df_X_train.csv")
df_X_test.to_csv("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\processed\\df_X_test.csv")

In [ ]:
y_train.to_csv("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\processed\\y_train.csv")
y_test.to_csv("C:\\Users\\yfawz\\OneDrive\\Desktop\\load_default_prediction\\data\\processed\\y_test.csv")